# Entrega 5.2 - Jugador de Damas Chinas (Q Learning, Redes Neuronales)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
El objetivo de esta tarea fue construir un jugador de **damas chinas** utilizando técnicas de aprendizaje por refuerzo, más concretamente, **Q Learning**. En este escenario se implementó una **red neuronal** para aproximar a la función de evaluación que ayuda al jugador a elegir su próxima jugada. Dicha función se ajusta a través de la experiencia jugando contra sí mismo o contra un jugador que elige su próximo movimiento de forma aleatoria.

En términos formales, los parámetros del problema se reducen a lo siguiente:
- **Tarea *T*:** Jugar una partida de Damas Chinas
- **Experiencia *E*:** Jugar contra un jugador aleatorio / Jugar contra sí mismo
- **Performance *P*:** Proporción de partidas ganadas

### 1.2. Entrega
***
La entrega de esta tarea consta de dos grandes componentes:
- **Informe** en formato de Jupyter Notebook (este informe).
- **Programa** simulador de juego y entrenamiento (y código fuente comentado).

El objetivo del informe es centralizar la información relativa a la construcción del modelo así como los datos surgidos de las distintas pruebas realizadas. Como agregado opcional, se adjuntan algunos scripts para ayudar a la lectura o probar en tiempo real ciertas funcionalidades. 

Por otra parte, el simulador ofrece una interfaz en consola que permite entrenar jugadores con múltiples configuraciones paramétricas, obtener datos y gráficas del entrenamiento y además poder enfrentarse a ellos luego de entrenados. También se agrega la posibilidad de cargar en memoria a los jugadores entrenados durante la experimentación, pudiendo comparar el rendimiento de cada uno, hacer que se enfrenten en un torneo, etc.

### 1.3. Formato
***
En las siguientes secciones se especifica el diseño del modelo, justificando la toma de decisiones a la hora de construirlo y detallando las estrategias o algoritmos que fueron implementados para la configuración paramétrica, junto a las métricas utilizadas en la evaluación. Luego, se detalla la metodología de experimentación y con la misma los resultados obtenidos para cada configuración y cada tipo de jugador contra el que se entrenó. Finalmente se agregan conclusiones respecto a los resultados obtenidos.

Como detalle importante, dado que la consigna de la actual entrega presenta ciertos puntos en común con  similitudes con la de la entrega anterior, se reutilizaron varios conceptos. A continuación se realiza una breve descripción del contenido de cada sección, especificando aquellas subsecciones que reutilizan contenido (las cuales son denotadas por un asterisco en su título), con el objetivo de facilitar una lectura que tenga en cuenta el informe anterior.

La **sección 2 (Diseño)** reutiliza los conceptos de las secciones **2.1 (Juego)** y **2.2. (Tablero)**. No obstante, la sección 2.2 agrega nuevas apreciaciones necesarias para el entendimiento del diseño del modelo, por lo que sólo se marcan con (*) aquellas subsecciones que no agregan información. Lo mismo sucede en las secciones **2.5 (Jugadores)** y **2.6. (Métricas)**, donde se determinan nociones similares al laboratorio 1, agregando nuevas apreciaciones correspondientes a este escenario.

La **sección 3 (Experimentación)** es completamente nueva, encarando la experimentación de una forma bastante distinta en comparación al primer laboratorio, principalmente por las características del modelo en este escenario, así como los nuevos conceptos aprendidos a lo largo del curso. Se 

La **sección 4 (Conclusiones)** cuenta con una nueva lista de conclusiones extraídas de la realización de esta tarea, por lo que no comparte nada salvo ciertas comparaciones comparaciones con la tarea anterior.

## 2. Diseño
***
En esta sección se detallan ciertas nociones técnicas en lo que refiere al juego de las damas chinas, la implementación dada del mismo y las características de diseño utilizadas para construir el modelo.

En lo que respecta a los modelos, se aprovecha para denotar cuales parámetros se mantuvieron fijos y cuales parámetros y características se decidieron variar a lo largo de la experimentación.

### 2.1. Juego (*)
***
El juego **Damas Chinas** cuenta con múltiples variantes y reglas distintas. A continuación se especifican las reglas de su forma básica, así como los cambios implementados en este escenario con el motivo de simplificar u optimizar la resolución del problema:

**Jugadores:**
> - Puede haber entre 2 y 6 jugadores, los cuales juegan todos contra todos.
> - Cada jugador cuenta con 10 piezas.
> - *Simplificación:* Se limita la participación a 2 jugadores, con el objetivo de simplificar el problema a representar.

**Tablero:**
> - El tablero tiene forma de estrella de 6 puntas, el cual cuenta con 121 casillas dispuestas en forma hexagonal.
> - Esto implica que cada celda cuenta con 6 celdas adyacentes, a diferencia de una grilla rectangular, donde cada celda tiene 9 celdas adyacentes.
> - Cada jugador puede mover sus piezas hacia las celdas adyacentes que no estén ocupadas, o saltar piezas (suyas o del oponente) siempre y cuando el salto sea siguiendo la recta que contiene la pieza que salta y la pieza saltada.
> - Cada jugador comienza con sus 10 piezas en una punta de la estrella.
> - *Simplificación:* Se limita el tablero a ser un rombo de 81 casillas, eliminando los 4 triángulos que se encuentran a los lados del tablero. Dicha simplificación se realiza ya que en un juego entre 2, la existencia de dichos triángulos no tiene relevancia y solo agrega complejidad al representar el tablero.

<div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/tableroOriginal.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 1 - Tablero original </label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/tableroSimple.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 2 - Tablero simplificado </label>
    </div>
</div>

**Objetivo:**
> - El objetivo de un jugador es depositar todas sus fichas en el triángulo opuesto a su triángulo de inicio.
> - El juego se termina cuando uno de los jugadores cumple dicho objetivo, no existiendo posibilidad de empate.
> - *Simplificación:* Debido a que es posible que un partido se extienda en el tiempo tanto como los jugadores lo deseen, se agrega la posibilidad de **empatar** una partida, declarándose dicho empate luego de pasada una cierta cantidad configurable de turnos (por defecto 300).

**Complejidad:**
> - A diferencia del Go y las fichas no coronadas en las damas, en las damas chinas toda pieza puede moverse hacia atrás, adicionalmente un jugador podría realizar el movimiento inverso al movimiento previo, causando un bucle de movimientos que no alteran el estado final del campo.
> - *Simplificación:* Debido a la enorme cantidad de movimientos posibles (muchos de estos contraproducentes a la victoria del jugador), se definió restringir movimientos atrás por parte de los jugadores, (salvo que no haya movimientos posibles hacia adelante). También se decidió restringir movimientos inversos (que dejen las piezas del jugador en una disposición idéntica al tablero en los últimos turnos) a los últimos tres movimientos por parte del jugador (salvo que no haya más movimientos posibles).

En resumen, se realizan **5 modificaciones** a las reglas básicas del juego:
- Se limita la cantidad de jugadores a 2.
- Se transforma el tablero de una estrella de 121 casillas a un rombo de 81 casillas.
- Se agrega la posibilidad de empatar si no se logra terminar el juego luego de cierta cantidad de turnos.
- Sólo se permiten movimientos hacia atrás si el jugador no tiene más posibles movimientos hacia adelante.
- Se restringen movimientos inversos a los últimos tres realizados por el jugador (salvo que no queden más posibles movimientos). 

### 2.2. Tablero
***
Como ya se mencionó en la anterior sección, el tablero de **Damas Chinas** tiene una estructura particular. A continuación se trata el problema subdividiendo en varias secciones:

#### 2.2.1. Representación física y lógica (*)
***
Al tratarse de un tablero con una _grilla hexagonal_, no existe ninguna estructura predeterminada en Python para almacenarlo y mantener el movimiento de fichas sencillo e intuitivo. De esta forma, se generaron dos formas de representar el tablero: una **lógica** (para mostrarlo y realizar cálculos) y una **física** (para almacenarlo).

La **representación lógica** consta de un rombo de 81 casillas dispuestas hexagonalmente (cada casilla tiene 6 adyacentes excepto las de los extremos) las cuales cuentan con un par de _coordenadas axiales_ para representarlas. Dicho sistema de coordenadas cuenta con 2 ejes como un sistema cartesiano, con la diferencia de que dichos ejes no son perpendiculares. Para obtener una referencia visual, correr **script 1** al final de la sección.

La **representación física**, por otra parte, consta de una matriz cuadrada 9x9 de 81 casillas dispuestas las cuales cuentan con un par de _coordenadas cartesianas_ para representarlas (una matriz común y corriente). Existe una _biyección_ entre las casillas en la representación lógica y en la física. Dicha biyección permite obtener las coordenadas axiales de un par de coordenadas cartesianas, existiendo una función inversa que permite hacer lo contrario. La biyección se compone de dos transformaciones lineales consecutivas:

1. **Traslación del centro:** Partiendo de la matriz, la cual cuenta con el centro en (0,0) en una esquina, se traslada dicho centro a la celda (4,4) (el centro exacto de la matriz cuadrada).
2. **Agregado de los extremos:** Partiendo de la matriz con centro trasladado, se sustituyen todas las coordenadas y que sean mayores a 4 o menores a -4 restándoles o sumándoles 9 respectivamente.

Sea $f: R^2 \rightarrow R^2$ la expresión que representa a la biyección, con $f_1$ y $f_2$ las anteriores transformaciones:

- $f(x,y)=f_2(f_1(x,y))$

- $f_1(x,y)=(x-4, -y+4)$

- $f_2(x,y)=$
    - $(x,y)$ $\forall$ $(x,y)$ $:$ $|y| < 4$
    - $(x,y-9)$ $\forall$ $(x,y)$ $:$ $y \geq 4$
    - $(x,y+9)$ $\forall$ $(x,y)$ $:$ $y \leq -4$

Para obtener una referencia visual, correr **script 1** al final de la sección.

#### 2.2.2. Representación de atributos
***
Se optó por trabajar con dos representaciones posibles para los atributos.
- **Métricas:** (Empleadas previamente en el laboratorio 1), estas mismas aportan un mayor grado de abstracción en la representación del tablero, esto permite acoplar tableros no necesariamente iguales en términos de la distribución de sus piezas pero sí en términos de "particularidades de la distribución de las mismas".
- **Tablero:** La disposición física de piezas en el tablero, denotando para cada celda si está vacia, tiene una pieza del jugador 1 o tiene una pieza del jugador 2.

A continuación se describe detalladamente ambas representaciones:

##### 2.2.2.1. Métricas (*)
***
Se determinaron **4 métricas**, las cuales deberían reflejar lineamientos generales sobre que tan bien o mal está jugando el jugador _i_, estas son:
1. **$A_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia el objetivo.** El objetivo de esta métrica es incitar al jugador _i_ a mover sus piezas al extremo opuesto del tablero.
2. **$B_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia la línea vertical que representa el centro del tablero.** El objetivo de esta métrica es incitar al jugador _i_ a no alejar sus piezas de la línea central vertical del tablero, ya que esto en general consume movimientos y desperdicia posibles cadenas de saltos.
3. **$C_i$** - **Suma de la cantidad de saltos posibles de cada pieza del jugador _i_.** El objetivo de esta métrica es incitar al jugador _i_ a generar cadenas de saltos para hacer avanzar sus piezas más rápidamente.
4. **$D_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia la casilla libre más cercana en el triángulo objetivo.** El objetivo de esta métrica es incitar al jugador _i_ a dirigirse al triángulo opuesto y rellenarlo con sus fichas. Se considera que esta métrica no sólo no es redundante con **$A_i$** , sino que además la complementa.

De esta forma, esta representación consta de un **Conjunto de 8 atributos** = $[A_1, B_1, C_1, D_1, A_2, B_2, C_2, D_2]$, el cual engloba las 8 métricas mencionadas anteriormente.

Finalmente, es importante detallar que los atributos obtenidos se **normalizan** pasando a estar en el intervalo $[0..1]$. Dicha normalización se implementa utilizando la **norma euclídea** y siguiendo la siguiente fórmula, siendo $x$ el vector de atributos, $x_i$ la coordenada _i_-ésima del mismo, $z$ el vector de atributos normalizado y $z_i$ la coordenada _i_-ésima del mismo:

$$z : z_i = \frac{x_i}{||x||}$$  

$$\forall i \in [0..length(x)]$$

##### 2.2.2.2. Tablero
***
Como los modelos a entrenar para este laboratorio constan de una red neuronal, se decidió considerar al tablero en sí como representación de los atributos.

De esta forma se considera al tablero como la matriz empleada en su representación, un rombo de dimensión 9x9 y este mismo se representa en un único vector de longitud 81, donde cada posición está vinculada a su respectiva celda en el tablero. Los posibles valores para cada celda son los siguientes:
- **1:** Si la celda es ocupada por una pieza del *Jugador 1*.
- **0:** Si la celda no es ocupada por ninguna pieza.
- **-1:** Si la celda es ocupada por una pieza del *Jugador 2*.

In [1]:
# A continuación se adjunta un script que permite comparar las distintas representaciones del tablero interactivamente, 
# extrayendo sus atributos y viendo como el mover piezas afecta a los mismos.
# Script 1 - Representación del tablero
import ntShowBoard

Box(children=(Label(value='Representación del tablero:', layout=Layout(margin='0px 16px 0px 0px')), ToggleButt…


                                (+4 O -8)

                            (+3 O -7)(+4 O -7)

                        (+2 O -6)(+3 O -6)(+4 O -6)

                    (+1 O -5)(+2 O -5)(+3 O -5)(+4 O -5)

                (+0 X -4)(+1 X -4)(+2 X -4)(+3 X -4)(+4 X -4)

            (-1 X -3)(+0 X -3)(+1 X -3)(+2 X -3)(+3 X -3)(+4 X -3)

        (-2 X -2)(-1 X -2)(+0 X -2)(+1 X -2)(+2 X -2)(+3 X -2)(+4 X -2)

    (-3 X -1)(-2 X -1)(-1 X -1)(+0 X -1)(+1 X -1)(+2 X -1)(+3 X -1)(+4 X -1)

(-4 X +0)(-3 X +0)(-2 X +0)(-1 X +0)(+0 X +0)(+1 X +0)(+2 X +0)(+3 X +0)(+4 X +0)

    (-4 X +1)(-3 X +1)(-2 X +1)(-1 X +1)(+0 X +1)(+1 X +1)(+2 X +1)(+3 X +1)

        (-4 X +2)(-3 X +2)(-2 X +2)(-1 X +2)(+0 X +2)(+1 X +2)(+2 X +2)

            (-4 X +3)(-3 X +3)(-2 X +3)(-1 X +3)(+0 X +3)(+1 X +3)

                (-4 X +4)(-3 X +4)(-2 X +4)(-1 X +4)(+0 X +4)

                    (-4 O +5)(-3 O +5)(-2 O +5)(-1 O +5)

                        (-4 O +6)(-3 O +6)(-2 O +6)

                            (-4 O +7)(-

Box(children=(Box(children=(Label(value='Movimiento de jugador 1:', layout=Layout(margin='0px 16px 0px 0px')),…

### 2.3. Modelo
***

De manera similar al laboratorio 1, el modelo implementado trabaja con conceptos de *aprendizaje por refuerzos*, orientándose por decisiones similares a las realizadas en el pasado (partidas anteriores) sobre la base de si estas mismas tuvieron un buen resultado (fueron recompensadas) o un mal resultado (fueron penalizadas).

El tipo de *aprendizaje por refuerzos* con el que se trabaja es **Q-Learning**, un algoritmo visto en el curso (el cual será más profundamente detallado en la brevedad), sin embargo una diferencia importante en la implementación del mismo es que representa la función **Q** mediante una red neuronal (en lugar de una tabla).

Es de esta forma que se determinó experimentar utilizando una red neuronal implementada por la clase *MLPRegressor* de la biblioteca [SKLearn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html), la cual procesa una entrada dada y retorna un posible valor para **Q**.

### 2.4. Algoritmo
***
Como fue mencionado anteriormente, el algoritmo implementado por los jugadores es una variación de *Q-Learning*. Utilizando dicho algoritmo, el jugador en cuestión determina que movimiento hacer en el tablero, consultando a su respectivo modelo por una *predicción* del valor de **Q** para cada uno de los tableros asociados a los posibles movimientos. Una vez estos están computados, se determina el movimiento que tenga asociado el mayor valor de **Q**(eligiendo uno aleatorio en caso de empate), ejecutándolo.

Este proceso se repite hasta que la partida llega a un final, es ahí cuando se pueden determinar con certeza cuales eran los correctos valores de **Q**, asignando $1$ a una victoria, $-1$ a una derrota y $-0.5$ a un empate, ajustando <b>Q</b> para los movimientos anteriores con un factor $\gamma = 0.9$.

Una vez se cuentan con los correctos valores de **Q** asociados a cada movimiento, estos datos son proporcionados a la red neuronal del modelo, para que entrene y reajuste minimizando el error en sus predicciones. Este proceso es repetido a lo largo de cierta cantidad de  partidas, en donde el resultado final es un jugador entrenado.

A continuación se detallan con mayor profundidad los algoritmos de entrenamiento y predicción que implementa la red neuronal en cuestión:

#### 2.4.1. Algoritmo de predicción
***
Para conseguir el valor de salida, las redes neuronales implementan *Forward Propagation (Propagación hacia adelante)*, también denominada *Feed-Forward*. Este algoritmo consiste en los siguientes pasos:
1. Los valores recibidos en la capa de entrada son multiplicados por los pesos. Un sesgo ("bias", constante) es agregado a la suma de las entradas por sus pesos (a modo de evitar valores nulos)
2. Cada neurona en la primer capa oculta recibe los distintos valores de la capa de entrada (dependiendo de los pesos y bias de la misma). Cada neurona tiene una función de activación que opera sobre los valores recibidos y da un valor entre 0 y 1 (inclusive) denotando la activación de la neurona.
3. Análogamente, las salidas de las neuronas de la primer capa oculta son multiplicadas por los respectivos pesos y se agrega un bias antes de ser procesadas por la capa siguiente. Este proceso se repite hasta que la capa de salida es alcanzada. Los valores calculados por la capa de salida son la salida retornada por el algoritmo.

El algoritmo de propagación hacia adelante consiste en esos tres pasos, sin embargo, la salida predecida no necesariamente es correcta, puediendo dar un valor erróneo. En tal caso es necesaria una corrección. El propósito del algoritmo de entrenamiento es hacer que las predicciones sean lo más exactas posibles. Para mejorar los resultados predecidos,a través de una fase de *Back Propagation*, los pesos de las distintas neuronas son reajustados de forma tal que la diferencia entre la salida predecida y la deseada sea lo más pequeña posible.

#### 2.4.2. Algoritmo de entrenamiento
***
Para entrenar, las redes neuronales implementan *Back Propagation (Propagación hacia atrás)*, un algoritmo que consiste en los siguientes pasos:
1. Se calcula el error cuantificando la diferencia entre la salida predecida y la salida deseada. A esta diferencia se le llama _"loss"_ o **costo**, y la función empleada para calcular la diferencia es llamada **función de costo**. Si bien existen múltiples variantes para la función de costo, SKLearn implementa la **función de costo cuadrático**.
2. Una vez el costo es calculado, el siguiente paso es minimizar el mismo (minimizando así el error). Para hacer esto, se emplea **descenso por gradiente estocástico (sgd)**, en donde se calculan las derivadas parciales de la función de costo respecto a los pesos y biases de la red, ya que por medio de las derivadas se puede determinar cual es la pendiente de la función de costo. Si la pendiente es positiva, el valor de los pesos puede ser reducido, en cambio, si la pendiente es negativa estos mismos pueden ser incrementados. Esto ajusta la función en una dirección opuesta al gradiente, reduciendo el error del modelo y eventualmente convergiendo a un mínimo local.

Un ciclo de _Forward Propagation_ y _Back Propagation_ es denominado un **epoch**. Este proceso continua hasta que un nivel razonable de _accuracy_ es alcanzado.

#### 2.4.3. Arquitectura
***
La arquitectura de una red neuronal es determinada por la disposición de neuronas a lo largo de sus capas y como estas están conectadas entre sí.

La arquitectura implementada por [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) (Multi-layer Perceptron Regressor), es una de las más básicas y tradicionales en el área de las redes neuronales y consiste en conectar cada una de las neuronas de la capa $n$ con cada una de las neuronas de la capa $n+1$, componiéndose de esta forma por una gran cantidad de parámetros (pesos y biases)

##### 2.4.3.1. Arquitectura - Capa de entrada
***
Dependiendo de como se represente el tablero, existen 2 opciones para determinar la cantidad de neuronas de la capa de entrada, siendo las mismas:
- **8 neuronas** - (Métricas): Se optó por determinar que los atributos de entrada para este modelo sean las métricas que dieron un mejor resultado para el laboratorio 1 (detalladas en la sección 2.2). De esta forma este modelo es compuesto por una red formada por 8 neuronas en su **capa de entrada**, en donde cada una recibe un vector de métricas y determina un valor para **Q** procesando las mismas.

- **81 neuronas** - (Tablero): Debido a que las redes neuronales manejan mejor mayores cantidades de información que las funciones lineales implementadas en el laboratorio 1, se determinó utilizar la representación entera del tablero como entrada. De esta forma este modelo es compuesto por una red formada por 81 neuronas en su *capa de entrada*, en donde cada neurona recibe un vector de cada celda del tablero, el cual contiene información sobre el estado de la misma (si se encuentra vacía, ocupada por una pieza del jugador 1 u ocupada por una pieza del jugador 2).

##### 2.4.3.2. Arquitectura - Capas ocultas
***
Dado que es muy difícil determinar que arquitectura de las capas ocultas dará un mejor resultado en una red neuronal para un problema dado, se decidió experimentar con posibles configuraciones de la misma, estas son:
- 1 capa oculta compuesta por 10 neuronas.
- 1 capa oculta compuesta por 100 neuronas.
- 2 capas ocultas cada una compuesta por 10 neuronas.
- 2 capas ocultas cada una compuesta por 100 neuronas.
- 3 capas ocultas cada una compuesta por 10 neuronas.
- 3 capas ocultas cada una compuesta por 100 neuronas.

A grandes rasgos, se decidió variar la cantidad de capas y la cantidad de neuronas de forma uniforme, con el objetivo de encontrar una relación (si la hay) entre el rendimiento del modelo y las capas ocultas implementadas.

##### 2.4.3.3. Arquitectura - Capa de salida
***
Como fue mencionado anteriormente, la cantidad de neuronas en la capa de salida es 1 para todos los modelos, esto es debido a que el rol que está cumpliendo la red neuronal del modelo en *Q-Learning* es aproximar la **función Q**.

#### 2.4.4. Parámetros
***
Existen múltiples parámetros para configurar una red neuronal de la clase *MLPRegressor*.Se decidió fijar los siguientes:
- Se trabaja con la flag ***warm_start*** = *True*, esto permite tener un entrenamiento iterativo en la red (a diferencia de un único entrenamiento determinado por un enorme dataset)
- Se trabaja con el ***solver*** = *sgd*, el cual implementa descenso estocástico por gradiente para aproximar la función.

##### 2.4.4.1. Parámetros - Función de activación
***
Para la función de experimentación se experimentó con tres de las cuatro posibles opciones de la biblioteca, para el parámetro ***activation***:
- **Logistic (Sigmoide):** $f(x) = \frac{1}{1 + e^{-x}}$
- **Tanh (Tangente hiperbólica):** $f(x) = tanh(x)$
- **ReLU (Rectified Linear Unit):** $f(x) = max(0,x)$

Debido a que no es trivial determinar que función de activación funciona mejor para un problema y modelo dados, se decidió experimentar con estos valores, los resultados se detallan en la sección 3.

##### 2.4.4.2. Parámetros -  Ratio de aprendizaje y enfriamiento
***
Por medio de observaciones empíricas realizadas en la construcción del algoritmo, se decidió que podía ser de valor incrementar el ratio de aprendizaje asignado por defecto en la documentación de *MLPRegressor*. Se decidió experimentar con los siguientes ratios de aprendizaje:
- **Constante:** con valor inicial 0.001.
- **Invscaling (Variable):** con valor inicial 0.1. El mismo es gradualmente reducido siguiendo la formula $learningRate = \frac{learningRateInicial}{t^{0.5}}$. Incrementando el valor de t con cada epoch.

### 2.5. Oponentes
***
El entrenamiento de los jugadores generados en este laboratorio se realizó jugando 500 partidas contra un oponente, los tipos de oponente son los siguientes:

#### 2.5.1. Jugador aleatorio (*)
***
**Funcionamiento:** Se define como **jugador aleatorio** a un programa que elige sus jugadas de forma aleatoria. Utilizando la biblioteca `random` de Python, el jugador aleatorio evalúa todos los posibles movimientos para cada una de sus fichas y elige uno de los mismos al azar.

**Motivo:** Puede resultar interesante evaluar la forma en la que el jugador aprende a entrenar al tener un oponente que no juega intentando ganar.

**Consideraciones al entrenar:** A diferencia del laboratorio anterior, decidimos extender la restricción aplicada al jugador de **prohibir movimientos hacia atrás** (a no ser no haya movimientos posibles hacia adelante), para lograr conseguir más valor al entrenar contra un jugador aleatorio. El motivo de esto es que en el *laboratorio 1* observamos que al no imponerse esta restricción, las partidas contra el jugador aleatorio no aportan valor, dado que es muy probable el jugador aleatorio deje piezas estancadas en su triangulo que luego terminen siendo bloqueadas por su oponente.

#### 2.5.2. Jugador previo (Red neuronal)
***
**Funcionamiento:** Se define como **jugador previo** a un programa que elige sus jugadas utilizando la misma función de evaluación que uso su oponente en el partido anterior. En otras palabras, es la versión del jugador que se está entrenando pero en la anterior iteración. 
Sin entrar en detalle, evalúa todos los tableros resultantes de todos los posibles movimientos para cada una de sus fichas y elige uno de la lista de movimientos con mejor evaluación (aleatoriamente).

**Motivo:** Puede ser una buena forma de automatizar el aprendizaje del juego contra un oponente que intenta ganar de la misma forma que el jugador a entrenar.

**Consideraciones al entrenar:** El enfrentar dos jugadores que realicen tácticas muy similares y reaccionen de igual manera representa un sesgo en el proceso de entrenamiento, dado que el jugador no será expuesto a grandes variaciones en la estrategia de juego.

#### 2.5.3. Jugador entrenado (Función lineal)
***
**Funcionamiento:** Se define como **jugador función lineal** al mejor jugador obtenido en el *laboratorio 1*.

**Motivo:** Es de gran interés evaluar nuestros jugadores entrenados en contra el resultado adquirido en el *laboratorio 1*, el cual en particular logró explotar las métricas del tablero lo suficiente como para ser un jugador bastante hábil en las damas chinas.

**Consideraciones:** Debido a que este jugador ya está entrenado, no se empleó para el entrenamiento de los jugadores en este laboratorio, sino que será empleado para la obtención de resultados y conclusiones en una de las etapas finales de la experimentación.

### 2.6. Evaluación
***
A continuación se detalla como se encaró la evaluación de los jugadores.

#### 2.6.1. Métodos
***
Para determinar cual de los jugadores tuvo un mejor rendimiento a lo largo de su entrenamiento se emplearon las dos métricas **_win rate_** y **_victory rate_** (detalladas en la siguiente subsección).

Es importante tener en cuenta que estas dos métricas sólo reflejan la calidad del jugador a lo largo de su entrenamiento, el cual es siempre contra un estilo de oponente fijo (si mismo o aleatorio). Por lo tanto no necesariamente reflejan la calidad del jugador (dado que un jugador con alto porcentaje de _win rate_ puede tener una muy mala peformance al enfrentarlo contra un oponente con estrategias nunca antes vistas por el jugador.

A raiz de esto último es que decidió evaluar la calidad de los jugadores por medio de un torneo, de esta forma podemos obtener métricas de los jugadores compitiendo entre sí. Esto es detallado en la siguiente sección (Experimentación).

#### 2.6.2. Métricas (*)
***
Se trabaja con las siguientes métricas:
- **Cantidad de Partidas Ganadas (_win rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas jugadas independientemente a su resultado, siguiendo la fórmula $$winRate = \frac{ganadas}{jugadas}$$
<br><br>
- **Ratio de Victorias (_victory rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas con algún resultado, es decir, no empatadas, siguiendo la fórmula $$victoryRate = \frac{ganadas}{ganadas + perdidas}$$
<br><br>
El motivo de tomar la segunda métrica además de la primera, es el hecho de que un jugador con más victorias que otro no es necesariamente mejor. Sea el escenario donde un **jugador 1** gana y pierde 50 partidas respectivamente y un **jugador 2** gana 25 y empata 75. Si bien **jugador 1** ganó el doble de partidas que **jugador 2**, perdió 50 partidas más. El motivo de los empates de **jugador 2** es indeterminado, por lo que no se puede decir quien de los dos es mejor, pero es importante tener en cuenta que **jugador 1 no es necesariamente mejor**.


## 3. Experimentación
***
En esta sección se detalla el conjunto de pruebas realizadas, agregándose observaciones pertinentes y comparaciones entre modelos.
A diferencia de la etapa anterior, se hace hincapié en como las pruebas influyeron en el proceso de construcción del modelo. Finalmente, luego de tener un diseño aparentemente libre de errores conceptuales, se comparan los rendimientos de distintas configuraciones paramétricas.

### 3.1. Metodología
***
Con el objetivo de evaluar las estrategias planteadas anteriormente y su efectividad, se probaron combinaciones de las mismas frente a distintos jugadores. A su vez, se buscó optimizar el proceso de extracción de mejores modelos lo máximo posible. Para ello, la exprimentación se encaró diviendola en las siguientes etapas:

1. **Recopilación y ajuste:** Debido a las diversas opciones a la hora de realizar el diseño del modelo, se tomó un enfoque de ensayo y error; para cada posible decisión se realizaron pruebas y en base a los resultados se tomaron o descartaron ciertas decisiones.
<br><br>

2. **Configuraciones paramétricas:** Luego de llegar a un modelo relativamente libre de errores (al menos de errores identificados), se realizaron pruebas para múltiples configuraciones paramétricas, con el fin de comparar cuales de ellas resultaron en mejores jugadores. En esta etapa se detallan los datos obtenidos. 
<br><br>

3. **Elección y comparación de representantes:** Habiendo generado una gran cantidad de jugadores con parámetros distintos, se procedió a analizar el rendimiento de cada uno de forma cuantitativa, con el fin de determinar aquellas configuraciones paramétricas que resultaron en jugadores con mejores resultados. Para mejorar la apreciación, se realizó un análisis cualitativo de . Habiendo realizado eso, se determinaron los **8 mejores jugadores** entreados contra un jugador aleatorio y contra sí mismos.
<br><br>

4. **Ponderación de representantes:** Utilizando los dos grupos de **8 jugadores** determinados en la sección anterior, se simuló un torneo para cada grupo, siguiendo una lógica similar a la de las eliminatorias de un campeonato de fútbol. El ganador de cada torneo se determinó como el mejor jugador de entre todos los modelos.
<br><br>

5. **Comparación con jugador lineal:** Utilizando a los mejores jugadores de la sección anterior, se simularon 100 partidas entre el mejor jugador del laboratorio 1 y el mejor del laboratorio 5, con el objetivo de determinar cual de los dos presenta un mejor desempeño jugando a las damas
<br><br>


### 3.2. Recopilación y ajuste
***
Antes de determinar cuales instancias evaluar en la configuración paramétrica, se observaron los resultados de variar ciertos parámetros de manera empírica, con el fin de evaluar si configuraciones de los mismos podían aportar a la configuración paramétrica o si su inclusión no iba a justificar el coste de evaluarlos exhaustivamente en el resto de la configuración.

Cabe destacar que al no tratarse de una experimentación exhaustiva (como la realizada en la configuración paramétrica), durante el proceso observación no fue pensado para mostrar resultados en la sección de experimentación, sino más como un artefacto interno.

En esta sección se mencionan cuales fueron esos parámetros y porque se decidió experimentar con un conjunto reducido de valores para los mismos (o directamente no experimentar con ellos, estableciendo su valor en el por defecto establecido por SKLearn).
- **Ratio de aprendizaje:** Si bien en la configuración paramétrica incluimos dos variantes para este valor (constante cuyo valor inicial es 0.0001 (sugerido por defecto en SKLearn) y variable (invscaling) con t = 0.5), originalmente se había considerado incluir valores más altos para el ratio de aprendizaje (de forma simialr al *laboratorio 1*). Esta idea fue descartada cuando observamos que en general dichos valores terminaban siendo perjudiciales para el entrenamiento (o terminaban no haciendo impacto).
- **Solver:** Para este parámetro se optó por fijar *descenso estocástico por gradiente (sgd)*, dado que fue el resaltado en el teórico. Otro solver que implementa una variación de sgd es *adam* (que además es el solver sugerido por defecto en *MLPRegressor*), sin embargo evaluamos que no iba a aportar a nuestro problema, dado que la documentación resalta que funciona bien para datasets de muy gran tamaño.
- **Power T:** Debido a que este parámetro aplica sólo para el ratio de aprendizaje *invscaling*, se decidió que no era de valor experimentar con el valor del mismo, optando por mantener su valor sugerido por defecto (0.5)

### 3.3. Configuraciones paramétricas
***
A continuación se adjuntan las distintas configuraciones paramétricas utilizadas al evaluar, sus motivaciones y sus resultados. Se realizan pequeñas observaciones sobre los mismos, quedando el análisis exhaustivo pendiente para la siguiente sección.

#### 3.3.1. Evaluaciones
***
Se generaron un total de 144 jugadores a evaluar, 72 fueron entrendos contra el *oponente aleatorio*, mientras que los otros 72 fueron entrenados contra el *oponente previo*. Las instancias se definen al variar los siguientes parámetros:
- Tipo de modelo (2 opciones: **Métricas como entrada**, **Tablero como entrada**)
- Función de activación (3 opciones: **Logistic (sigmoide)**, **Tanh**, **ReLU**)
- Arquitectura de las capas ocultas (6 opciones: **(10)**, **(100)**, **(10, 10)**, **(100, 100)**, **(10, 10, 10)**, **(100, 100, 100)**)
- Ratio de aprendizaje (2 opciones: **Constante, valor inicial = 0.001)**, **Invscaling (variable), valor inicial = 0.1**)

#### 3.3.2. Resultados
***
Debido a la gran magnitud de resultados generados, el análisis de los mismos es abordado en la siguiente sección, en la cual se divide el análisis de los resultados obtenidos cuantitativa y cualitativamente.

### 3.4. Elección y comparación de representantes
***

#### 3.4.1. Análisis cuantitativo
***
A continuación se brindan graficas de barras tanto para el Win Rate como para el Victory Rate.

En lugar de intentar análizar el impácto de todos los parámetros las gráficas muestran la distribución de Win Rate y Victory Rate, para subconjuntos que dividen a los jugadores según si fueron entrenados versus un *oponente aleatorio* o un *oponente previo* y según si implementan un modelo de *métricas como entrada* o *tablero como entrada*.

A partir de las distribuciones para estos conjuntos, observaciones son planteadas.

##### 3.4.1.1. Análisis del Win Rate
***

In [ ]:
# Graficas de Win Rate para todos los jugadores.
%matplotlib inline
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from utils.const import CompareOps
from ntCompare import plot

plot(CompareOps.WIN_RATE)

A partir de las gráficas se observa que:
- Los modelos que reciben métricas como entradas lograron alcanzar consistentamente un mejor Win Rate en contra de ambos  tipos de oponentes.
- Los modelos que reciben el tablero como entrada tuvieron resultados de muy baja calidad al enfrentarse al *oponente aleatorio*.
- Para ambos modelos el resto de los parámetros impactó fuertemente en sus resultados, sin embargo se observa que los modelos que reciben el tablero como entrada tienen una distribución un tanto más uniforme de sus resultados que los modelos que reciben métricas como entrada.
- Para ambos tipos de modelos se observa que no necesariamente una configuración que tenga buenos resultados contra un tipo de oponente los tendrá con el otro.
    - Se destaca el jugador N° 26 con tablero. Alcanzó el Win Rate más alto vs un oponente aleatorio pero también el más bajo contra si mismo (Empatando todas las partidas).
    - Observaciones análogas pueden realizarse para múltiples jugadores.

##### 3.4.1.2 Análisis del Victory Rate
***

In [ ]:
# Graficas de Victory Rate para todos los jugadores.
%matplotlib inline
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from utils.const import CompareOps
from ntCompare import plot

plot(CompareOps.VICTORY_RATE)

A partir de las gráficas y vinculando el Victory Rate al Win Rate de la sección anterior se observa que:
- Tanto los jugadores que fueron entrenados en contra del *oponente previo* y recibian métricas como entrada, así como los jugadores que fueron entrenados en contra del *oponente aleatorio* y recibian el tablero como entrada, tienen una distribución casi idéntica de Win Rate vs Victory Rate entre si, esto es a causa de que la cantidad de empates ocurridos en el entrenamiento de estos jugadores fue casi nula.

#### 3.4.2. Análisis cualitativo
***

A continuación se brindan graficas que muestran el impacto en la curva del Win Rate promedio para subconjuntos determinados por cada uno de los valores de un posible parámetro en la configuración.

A diferencia de la sección anterior, esta sección si se concentra en determinar (para cada uno de los subconjuntos definidos en la sección anterior) cuales configuraciones del modelo *MLPRegressor* dieron en general un mejor resultado para el algoritmo.

##### 3.4.2.1 Análisis del impacto de la arquitectura de capas ocultas en base al Win Rate
***

In [ ]:
# Curvas de impacto de la arquitectura de capas ocultas para el Win Rate promedio de los jugadores.
%matplotlib inline
from utils.const import CompareOps
from ntCompare import plot

plot(CompareOps.HIDDEN_LAYERS)

A partir de las gráficas se observa que:
- Los jugadores entrenados en contra del *oponente previo* tuvieron una mejor performance para dos capas ocultas.
    - Una observación similar no se puede dar para los jugadores entrenados en contra del *oponente aleatorio*, dado que los que recibian el tablero como entrada tuvieron mejor perfomance para una capa oculta, mientras que los que recibian métricas como entrada tuvieron mejor performance para tres.
- Contar con 100 neuronas en cada capa oculta siempre dio mejores resultados que contar con 10 neuronas en cada capa oculta.

##### 3.4.2.2 Análisis del impacto de la función de activación en base al Win Rate
***

In [ ]:
# Curvas de impacto de la función de activación para el Win Rate promedio de los jugadores.
%matplotlib inline
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from utils.const import CompareOps
from ntCompare import plot

plot(CompareOps.ACTIVATION)

A partir de la gráfica se observa que:
- La función Sigmoide dio un muy mal resultado para todos los conjuntos de jugadores salvo los que reciben métricas de entrada y fueron entrenados contra el *oponente previo*, en donde la función Sigmoide tuvo drásticamente la mejor performance.
- La función ReLU tuvo la mejor performance para los jugadores que reciben el tablero como entrada.
- La función TanH tuvo muy buena performance para los jugadores que se enfrentaron al *oponente previo*, sin embargo dio resultados de muy mala calidad para los jugadores que fueron enfrentados al *oponente aleatorio*.

##### 3.4.2.3 Análisis del impacto del ratio de aprendizaje en base al Win Rate
***

In [ ]:
# Curvas de impacto del ratio de aprendizaje para el Win Rate promedio de los jugadores.
%matplotlib inline
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from utils.const import CompareOps
from ntCompare import plot

plot(CompareOps.LEARNING_RATE)

A partir de la gráfica se observa que:
- El ratio de aprendizaje variable con valor inicial en 0.1 tuvo mejor performance que constante para todos los jugadores salvo los que recibian métricas como entrada y fueron enfrentados al *oponente previo* (en este caso no pareció haber afectado la calidad de los resultados).

#### 3.4.3. Representantes
***

Se dividieron los jugadores a evaluar según si habian sido entrenados contra el *oponente aleatorio* o contra el *oponente previo*.

Se eligieron como representantes de cada conjunto a los 8 jugadores con más **_win rate_**, estos dos conjuntos de jugadores pasarán a la siguiente etapa de ponderación de representantes (el torneo).

##### 3.4.3.1 Mejores jugadores entrenados contra el *oponente previo*
**TODO: Sacar titulos de las tablas?)**

<table>
    <tr>
        <th>Índice</th>
        <th>Modelo</th>
        <th>Arquitectura de las capas ocultas</th>
        <th>Función de activación</th>
        <th>Ratio de aprendizaje</th>
        <th>Partidas ganadas</th>
    </tr>
    <tr>
        <td>17</td>
        <td>Tablero</td>
        <td>(100, 100)</td>
        <td>TaNH</td>
        <td>Constante</td>
        <td>93%</td>
    </tr>
    <tr>
        <td>61</td>
        <td>Métricas</td>
        <td>(100, 100, 100)</td>
        <td>Sigmoide</td>
        <td>Constante</td>
        <td>93%</td>
    </tr>
    <tr>
        <td>57</td>
        <td>Métricas</td>
        <td>(10, 10)</td>
        <td>ReLU</td>
        <td>Constante</td>
        <td>92%</td>
    </tr>
    <tr>
        <td>42</td>
        <td>Métricas</td>
        <td>(100)</td>
        <td>TaNH</td>
        <td>Variable</td>
        <td>90%</td>
    </tr>
    <tr>
        <td>48</td>
        <td>Métricas</td>
        <td>(10)</td>
        <td>TaNH</td>
        <td>Variable</td>
        <td>89%</td>
    </tr>
    <tr>
        <td>45</td>
        <td>Métricas</td>
        <td>(10)</td>
        <td>ReLU</td>
        <td>Constante</td>
        <td>88%</td>
    </tr>
    <tr>
        <td>49</td>
        <td>Métricas</td>
        <td>(100, 100)</td>
        <td>Sigmoide</td>
        <td>Constante</td>
        <td>87%</td>
    </tr>
    <tr>
        <td>54</td>
        <td>Métricas</td>
        <td>(100, 100)</td>
        <td>TaNH</td>
        <td>Variable</td>
        <td>84%</td>
    </tr>
    <caption>Mejores jugadores entrenados contra el <i>oponente previo</i></caption>
</table>

##### 3.2.2.2 Mejores jugadores entrenados contra el *oponente aleatorio*

<table>
    <tr>
        <th>Índice</th>
        <th>Modelo</th>
        <th>Arquitectura de las capas ocultas</th>
        <th>Función de activación</th>
        <th>Ratio de aprendizaje</th>
        <th>Partidas ganadas</th>
    </tr>
    <tr>
        <td>51</td>
        <td>Métricas</td>
        <td>(100, 100)</td>
        <td>ReLU</td>
        <td>Constante</td>
        <td>99%</td>
    </tr>
    <tr>
        <td>50</td>
        <td>Métricas</td>
        <td>(100, 100)</td>
        <td>Sigmoide</td>
        <td>Variable</td>
        <td>93%</td>
    </tr>
    <tr>
        <td>44</td>
        <td>Métricas</td>
        <td>(10)</td>
        <td>Sigmoide</td>
        <td>Variable</td>
        <td>83%</td>
    </tr>
    <tr>
        <td>63</td>
        <td>Métricas</td>
        <td>(100, 100, 100)</td>
        <td>ReLU</td>
        <td>Constante</td>
        <td>82%</td>
    </tr>
    <tr>
        <td>54</td>
        <td>Métricas</td>
        <td>(100, 100)</td>
        <td>ReLU</td>
        <td>Variable</td>
        <td>81%</td>
    </tr>
    <tr>
        <td>26</td>
        <td>Tablero</td>
        <td>(100, 100, 100)</td>
        <td>Sigmoide</td>
        <td>Variable</td>
        <td>78%</td>
    </tr>
    <tr>
        <td>37</td>
        <td>Métricas</td>
        <td>(100)</td>
        <td>Sigmoide</td>
        <td>Constante</td>
        <td>77%</td>
    </tr>
    <tr>
        <td>43</td>
        <td>Métricas</td>
        <td>(10)</td>
        <td>Sigmoide</td>
        <td>Constante</td>
        <td>76%</td>
    </tr>
    <caption>Mejores jugadores entrenados contra el <i>oponente aleatorio</i></caption>
</table>


### 3.5. Ponderación de representantes
***
Las reglas del torneo son simples, se itera por cada uno de los jugadores participandes enfrentandolo contra los otros 7 contrincantes. Generando de esta forma 56 partidas (en las que se enfrentan un total de dos veces cada jugador).

Cada jugador obtiene 3 puntos por ganar, 1 punto por empatar y 0 puntos por perder. Al final el programa imprime una tabla de puntuación que registra las victorias de cada participante.

A continuación se adjunta un script que permite simular el torneo entre los jugadores con más Win Rate para cada conjunto.

Dada la aleatoriedad del torneo, luego de ese script se adjunta otro para cargar y mostrar la tabla que obtuvimos como resultado.

In [ ]:
# Torneo
%matplotlib inline
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import evaluation.evaluator as evaluator
import processing.archiver as archiver
import evaluation.tourney as tourney
import utils.gui as gui
from utils.const import PlayerType

# Comente y descomente según el conjunto de jugadores para participar en el torneo
playerType = PlayerType.TRAINED_SELF
#playerType = PlayerType.TRAINED_RANDOM

# Se cargan los jugadores
if playerType == PlayerType.TRAINED_SELF:
    players = archiver.loadMassive('self', True)
else:
    players = archiver.loadMassive('random', True)

# Se agarran los 8 mejores
tourneyPlayers = evaluator.getBestNeuralNetworks(players)
tourneyPlayers = tourneyPlayers[:8]
indexes = []
for player in tourneyPlayers:
    indexes.append(players.index(player))
for p, index in zip(tourneyPlayers, indexes):
    p['playerID'] = index + 1

# Se ejecuta el torneo
table = tourney.simulateTourney(tourneyPlayers)
gui.printTourneyResults(table, tourneyPlayers)

In [1]:
# Script para cargar los resultados del torneo obtenidos por nosotros:
%matplotlib inline
import os
import sys
import pickle
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import utils.gui as gui
from utils.const import PlayerType

# Comente y descomente según el conjunto de jugadores a los que mostrar su resultado
playerType = PlayerType.TRAINED_SELF
# playerType = PlayerType.TRAINED_RANDOM

if playerType == PlayerType.TRAINED_SELF:
    filename = 'results/tourney_trained_self'
else:
    filename = 'results/tourney_trained_random'

pickle_in = open(filename,'rb')
tourneyResults = pickle.load(pickle_in)

gui.printTourneyResults(tourneyResults['table'], tourneyResults['tourneyPlayers'])


-> RESULTADOS DEL TORNEO:

--> JUGADORES:

Jugadores actuales:

-> 17 - Modelo: Red Neuronal
--> Metadatos:
    Entrenado VS: PlayerType.TRAINED_SELF
    Tiempo de entrenamiento: 309.4432108402252 segundos
    Cantidad de iteraciones: 100
    Cantidad máxima de turnos por juego: 300
--> Parámetros:
    Representación del tablero: InputLayerTypes.BOARD
    Cantidad de capas ocultas: 2
    Cantidad de neuronas por capa oculta: 100
    Función de activación: ActivationFunctions.TANH
    Ratio de aprendizaje: ('constant', 0.001)
--> Resultados:
    Cantidad de partidas ganadas, perdidas, empatadas: [93, 6, 1]
    Porcentaje de partidas ganadas: 0.93

-> 61 - Modelo: Red Neuronal
--> Metadatos:
    Entrenado VS: PlayerType.TRAINED_SELF
    Tiempo de entrenamiento: 1089.3696110248566 segundos
    Cantidad de iteraciones: 100
    Cantidad máxima de turnos por juego: 300
--> Parámetros:
    Representación del tablero: InputLayerTypes.METRICS
    Cantidad de capas ocultas: 3
    Cantidad de neu

### 3.6. Comparación con jugador lineal
***
Se enfrentó cada uno de los ganadores de los torneos con el mejor jugador lineal obtenido en el laboratorio 1.

Estos duelos constaron de 100 partidas en las cuales ambos jugadores podían aprender retroalimentativamente de cada uno. Se definió cargar como jugador 2 al jugador entrenado en este laboratorio para darle la ventaja de jugar primero.

No se adjuntó un script de los mismos, pero pueden recrearse con los siguientes pasos:
- Cargar modelo -> 2. Carga masiva -> (random | self)
- Cargar modelo -> 1. Carga única -> jugador_lab1
- Entrenar modelo -> 3. Vs otra IA -> 73 (lab1), (54 | 49) -> Proseguir manualmente con el resto de opciones de entrenamiento

ACÁ VA UNA IMAGEN DE LOS RESULTADOS CONTRA LINEAL PARA CADA JUGADOR

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia1.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Partidas ganadas para cada modelo entrenado de la instancia 1</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia2.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Partidas ganadas para cada modelo entrenado de la instancia 2</label>
    </div>
</div>


### 3.7. Script para jugar
***

A continuación se adjunta un script que permite jugar partidas de damas chinas contra los dos mejores jugadores y el oponente del laboratorio 1.

In [ ]:
# Script 2 - Lector vs Jugadores
import ntPlay

## 4. Conclusiones
***

#### 4.1. Respecto al Modelo
***
A

#### 4.2. Respecto a los Parámetros
***
A

#### 4.3. Respecto a los Resultados
***
A

#### 4.4. Respecto a los Oponentes
***
A

#### 4.5. Posibles mejoras
***
A:
- A
- A